In [7]:
import qubic.toolchain as tc
import qubic.rpc_client as rc
import qubitconfig.qchip as qc
from distproc.hwconfig import FPGAConfig, load_channel_configs
import numpy as np
import matplotlib.pyplot as plt

# Play a simple pulse

## Load configuration information

1) define FPGA config; this has timing information for the scheduler. For now it is fine to use the following hardcoded config TODO for tutorial: put these parameters somewhere in the repo so it's not hardcoded for the demo

In [8]:
fpga_config = FPGAConfig(**{'fpga_clk_period': 2.e-9, 
                            'alu_instr_clks': 5, 
                            'jump_cond_clks': 5, 
                            'jump_fproc_clks': 5, 
                            'pulse_regwrite_clks': 3})

2. load channel configs, which assign named output channels to physical DAC outputs (or readout downconversion channels) and configure signal generator parameters. (see [Understanding Channel Configuration](https://gitlab.com/LBL-QubiC/software/-/wikis/Understanding-Channel-Configuration) for details)


In [9]:
channel_configs = load_channel_configs('channel_config.json')

example channel config for named output 'Q0.qdrv' (qubit drive channel for physical qubit 'Q0'): 

`ChannelConfig(core_ind=7, elem_ind=0, elem_params={'samples_per_clk': 16, 'interp_ratio': 1}, _env_mem_name='qdrvenv{core_ind}', _freq_mem_name='qdrvfreq{core_ind}', _acc_mem_name='accbuf{core_ind}')`

The physical output channel is given by `core_ind` and `elem_ind`, while signal generator params are given by `elem_params`.


## Define the pulse sequence

QubiC circuits (or "programs") are defined as lists of dictionary. Each dictionary is a pulse command, program statement, or timing construct

In [5]:
circuit = [
    #play a single pulse on the Q0 drive channel
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.334704954261188, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}, 
    
]

The default library of envelope functions is given in <link>. Alternatively, a numpy array of samples can be provided. A custom envelope library can also be passed in during the compile stage.

## Compile and Assemble

Compile the program. Since we don't have any references to gates, we can pass None to the qchip argument. The output of the compile stage is a distributed processor assembly program, which consists of initialization/termination statements, as well as a list of scheduled pulses for each core. Our program only uses one processor core, scoped to channels `('Q0.qdrv', 'Q0.rdrv', 'Q0.rdlo')`.

In [6]:
compiled_prog = tc.run_compile_stage(circuit, fpga_config, None)
compiled_prog.program

{('Q0.qdrv', 'Q0.rdrv', 'Q0.rdlo'): [{'op': 'phase_reset'},
  {'op': 'pulse',
   'freq': 4944383311,
   'phase': 0,
   'amp': 0.334704954261188,
   'env': {'env_func': 'cos_edge_square',
    'paradict': {'ramp_fraction': 0.25, 'twidth': 2.4e-08}},
   'start_time': 5,
   'dest': 'Q0.qdrv'},
  {'op': 'done_stb'}]}

Run the assembler to convert the above program into machine code that we can load onto the FPGA or gateware simulation:

In [7]:
asm_prog = tc.run_assemble_stage(compiled_prog, channel_configs)
asm_prog

Outline for the rest of this:
1. show pulse on Q0 and Q1
2. introduce timing constaints (delay and barrier), and have them construct particular sequences
3. demonstrate multiplexed readout (maybe demod also?)
4. basic branching demo
5. introduce looping/variables. task: generate n pulses with m varying amplitudes
6. simple calibration sequences, like T1, T2, rabi, etc

Send circuit to simulator and plot output:

## Multiple Pulses

Multiple pulses on the same channel are played sequentially. Pulses on separate channels are scheduled in parallel; i.e. each pulse is played as soon as it's output channel is available. 

Excercises:
  1. Try playing two or more pulses on the same channel
  2. Add a pulse or two to Q1.qdrv

In [2]:
circuit = [
    #play two pulses on the Q0 drive channel
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.3, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}, 
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}
    
]

In [ ]:
circuit = [
    #play two pulses on the Q0 drive channel
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.3, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}, 
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'},
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q1.qdrv'}
    
]

### Timing Constraints

Delays and barriers can be used to control when pulses are played relative to one another. Exercises:
  1. Play a pulse on Q1 after both pulses on Q0
  2. Apply a delay between the two Q0 pulses
  
Extrass: 

3. Scoped delays and barriers -- play the following pulse sequence:

In [4]:
circuit = [
    #play two pulses on the Q0 drive channel
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.3, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}, 
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'},
    
    {'name': 'barrier'}, 
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q1.qdrv'}
    
]

In [6]:
circuit = [
    #play two pulses on the Q0 drive channel
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.3, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'}, 
    
    {'name': 'delay', 't': 500.e-9},
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q0.qdrv'},
    
    {'name': 'barrier'}, 
    
    {'name': 'pulse', 'phase': 0, 'freq': 4944383311, 'amp': 0.6, 'twidth': 2.4e-08,
     'env': {'env_func': 'cos_edge_square', 'paradict': {'ramp_fraction': 0.25}},
     'dest': 'Q1.qdrv'}
    
]

## Gates

Calibrated gate information can be stored in a json file (example: qubitcfg.json) and referenced in QubiC circuits. Each gate consists of a list of a list of pulses

### Example of X90 gate on Q0

### Example of CNOT gate, consisting of a composite list of pulses

In [11]:
circuit = [
    # this circuit plays calibrated X90 gates
    {'name': 'X90', 'qubit': 'Q0'},
    {'name': 'X90', 'qubit': 'Q1'}]

In [12]:
# load in configuration from qubitcfg.json
qchip = qc.QChip('qubitcfg.json')

In [13]:
# inspect a gate
qchip.gates['Q0X90'].cfg_dict

[{'freq': 'Q0.freq',
  'phase': 0.0,
  'dest': 'Q0.qdrv',
  'twidth': 2.4e-08,
  't0': 0.0,
  'amp': 0.11222212331696187,
  'env': [{'env_func': 'cos_edge_square',
    'paradict': {'ramp_fraction': 0.25}}]}]

In [14]:
# link the qchip configuration at compile time
compiled_prog = tc.run_compile_stage(circuit, fpga_config, qchip)